# Feature Selection

This notebook is trying out different strategies for feature selection based on sklearn as well as a few neural approaches.

## Dependencies

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


## Dataset

Download dataset to parent data directory.

In [ ]:
!if [ ! -f ../data/dorothea.zip ]; then wget -P ../data/ https://archive.ics.uci.edu/static/public/169/dorothea.zip && unzip ../data/dorothea.zip -d ../data/; fi

In [ ]:
class DataPreprocessor:
    def __init__(self, features_file, targets_file):
        self.features_file = features_file
        self.targets_file = targets_file
        self.df = None
    
    def _preprocess(self):
        data = []
        with open(self.features_file, 'r') as f:
            for line in f:
                active_features = line.strip().split()
                data.append(pd.Series({int(feature): 1 for feature in active_features}))
        features = pd.concat(data, axis=1).T.fillna(0).sort_index(axis=1)
        targets = pd.read_csv(self.targets_file, header=None, names=["target"])
        self.df = pd.concat([features, targets], axis=1)
    
    def __call__(self):
        self._preprocess()
        return self.df

In [ ]:
wrangler = DataPreprocessor('../data/DOROTHEA/dorothea_train.data', '../data/DOROTHEA/dorothea_train.labels')

In [ ]:
df = wrangler()

In [ ]:
df.head()

In [ ]:
X, y = df.drop('target', axis=1), df['target']

# Feature Selection Algorithms

## Univariate Feature Selection

Selecting features using univariate statistical tests of teh relationship between each feature and the target variable.

### Variance Threshold

Let's see a histogram of the variance of each feature.

In [ ]:
variances = X.var()

plt.hist(variances, bins='auto', log=True)
plt.title('Histogram of Variances')
plt.xlabel('Variance')
plt.ylabel('Frequency')
plt.show()

In [ ]:
log_variances = np.log(variances + 1e-9)

plt.hist(log_variances, bins='auto', log=True)
plt.title('Histogram of Log Variances', )
plt.xlabel('Log Variance')
plt.ylabel('Frequency')
plt.show()

In [ ]:
plt.hist(variances, bins='auto', density=True, cumulative=True, histtype='step', alpha=0.8)
plt.title('CDF of Variances')
plt.xlabel('Log Variance')
plt.ylabel('Cumulative Probability')
plt.grid(True)

# Calculate the median of variances
median = np.median(variances)

# Plot the median as a dotted line
plt.axvline(median, color='b', linestyle='dotted', linewidth=2, label=f'Median Variance: {median:.2f}')

plt.legend()
plt.show()

In [ ]:
plt.boxplot(variances)
plt.title('Boxplot of Variances')
plt.ylabel('Variance')
plt.grid(True)
plt.show()

In [ ]:
plt.hist(log_variances, bins='auto', density=True, cumulative=True, histtype='step', alpha=0.8)
plt.title('CDF of Log Variances')
plt.xlabel('Log Variance')
plt.ylabel('Cumulative Probability')
plt.grid(True)

# Calculate the median of log_variances
median = np.median(log_variances)

# Plot the median as a dotted thick blue line
plt.axvline(median, color='b', linestyle='dotted', linewidth=2, label=f'Median Variance: {median:.2f}')

plt.legend()
plt.show()

In [ ]:
plt.boxplot(variances)
plt.title('Boxplot of Variances')
plt.ylabel('Variance')
plt.grid(True)
plt.show()

In [ ]:
from sklearn.feature_selection import VarianceThreshold

variance_cutoff = 0.01
selector = VarianceThreshold(threshold=variance_cutoff)
selected_features = selector.fit_transform(X)

### 